In [12]:
import numpy as np 
import math
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import datasets
import scipy.stats as st

In [28]:
def tune_bb(algo, X, y, 
            regularization="Dropout", M=10, 
            c=None, K=5, criterion="MSE"):

    """function to automatically tune blackbox regression model
    
    Parameters:
    -----------

    algo : callable
        A learning algorithm that takes as input a matrix X in R nxp
        and a vector of responses Y in Rn and returns a function that
        maps inputs to outputs. Must have methods like .fit() and .predict()
    X : array-like of shape (n,p)
        training data X in R nxp
    y : array-like of shape (n,)
        training labels, Y, in Rn
    regularization : str, default="Dropout"
        regularization method, can be any of "Dropout",
        "NoiseAddition", or "Robust"
    M : int, default=
        A positive integer indicating the number of Monte Carlo
        replicates to be used if the method specified is Dropout or 
        NoiseAddition
    c : default=None
        A vector of column bounds to be used if method specified is "Robust"
    K : int, default=5
        A positive integer indicating the number of CV-folds to be used to 
        tune the amount of regularization, e.g., K = 5 indicates five-fold CV
    criterion : str, default="MSE"
        A criterion to be used to evaluate the method that belongs to the set 
        {MSE, MAD} where MSE encodes mean square error and MAD encodes mean
        absolute deviation.

    Returns:
    -----------
    tuned_mode : callable 
        A tuned predictive model that optimizes the specific criterion using 
        the specified method

    Example:
    -----------
    >>> tune_bb()
    >>> 
    """
    
    # statements here to ensure model has the methods we need to tune it
    assert hasattr(algo, "fit"), "model object must have .fit() method"
    assert hasattr(algo, "predict"), "model object must have .predict() method"

    if criterion == "MSE":
        criterion = "neg_mean_squared_error"
    elif criterion == "MAE":
        criterion = "neg_mean_absolute_error"
    else:
        raise ValueError("Please input either MAE or MSE for criterion.")
    
    if regularization == "Dropout":
        # dropout notes: topic 1 p. 30 
        # draw Z matrix
        Z = np.random.binomial(1,0.5,size=X.shape)
        dropout_matrix = Z * X 
    elif regularization == "NoiseAddition":
        print('do something')
    elif regularization == "Robust":
        #Assertion: we don't actually have any reasonable bounds to argue for robust regression here
        #We could take a sample of many matrix M and then choose 1 based on a criteria to fit our regression
        #If we do this iteratively with changing our c bounds, we can theoretically narrow down our bounds wrt the MSE/MAE
        tol = False #Are we in our tolerance range
        toler = 0.001
        wts = [x/2 for x in c] #Initial weights are going to be c/2
        oerror = np.inf
        merror = np.inf
        itera = 0
        while not tol:
            print(itera, merror, wts)
            #Create a bunch of matrices and choose the best by some score
            maxmatrix = None
            maxnorm = -np.inf
            for i in range(1000):
                matrix = np.random.rand(X.shape[0], X.shape[1])
                for m in range(matrix.shape[1]):
                    matrix[:,m] = (wts[m] / np.linalg.norm(matrix[:,m], 2)) * matrix[:,m]
                fnorm = np.linalg.norm(matrix, 2) #The criteria I'm using here is the two-norm
                if fnorm > maxnorm:
                    maxnorm = fnorm
                    maxmatrix = matrix
            new_X = X + maxmatrix #We add the permuted matrix to our design matrix
            
            #kfold cross validation
            errors = np.abs(cross_val_score(algo, new_X, y, cv=K, scoring=criterion))
            merror = np.mean(errors)
            if abs(oerror - merror) > toler:
                print(abs(oerror - merror))
                oerror = merror #Save the current error for the next iteration
                #Set our new weights for the next iteration
                wts = np.minimum(wts + (np.random.normal(size = len(wts)) * math.exp(-itera)), c) 
                itera += 1
            else:
                tol = True
        
        #Once we have our best c bounds, let's use them exactly to construct the best model
        maxmatrix = None
        maxnorm = -np.inf
        for i in range(10000):
            matrix = np.random.rand(X.shape[0], X.shape[1])
            for m in range(matrix.shape[1]):
                matrix[:,m] = (wts[m] / np.linalg.norm(matrix[:,m], 2)) * matrix[:,m]
            fnorm = np.linalg.norm(matrix, 2) #The criteria I'm using here is the two-norm
            if fnorm > maxnorm:
                maxnorm = fnorm
                maxmatrix = matrix
        new_X = X + maxmatrix #We add the permuted matrix to our design matrix
        
        model = algo
        tuned_mode = model.fit(new_X, y)
        #print(tuned_mode.coef_)
        return tuned_mode
    
    else: 
        raise ValueError('Please input one of of "Dropout", "NoiseAddition", or "Robust"')



In [29]:
# matrices to play around with and feed into function 
# NO train test split here, just full data
X, y = datasets.load_iris(return_X_y=True)

# test here, maybe start testing with Ridge()
tune_bb(Ridge(),
        X,
        y,
        regularization="Robust",
        c = [2,2,1,1.5],
        criterion="MAE")

0 inf [1.0, 1.0, 0.5, 0.75]
inf
1 0.209957020924517 [ 0.98875363  0.83236253 -0.78772712  0.8983743 ]
[-0.11150889 -0.04352956  0.22879664  0.60848334]
[-0.11291719 -0.03549502  0.25856824  0.53864292]


Ridge()

### Scratchwork below

In [21]:
# maybe something like this if they all follow the .fit() syntax? 
def model_test(model,X,y):
    return model.fit(X,y)

In [22]:
# this passes the model in ?
model_test(LinearRegression(), X, y).coef_

array([-0.11190585, -0.04007949,  0.22864503,  0.60925205])

In [23]:
model_test(Ridge(), X, y).coef_ # niceeeeeeeeee

array([-0.11346491, -0.03184254,  0.25936799,  0.53764103])

In [20]:
model_test(Lasso(), X, y).coef_

array([ 0.57894737, -0.        , -0.        ])

In [ ]:
# ridge tuning parameter, for example
# expand this later
parameter_set = np.linspace(0,10,11) # 0 to 10

cv_score_set = []
# begin cross validation
for alpha in parameter_set:

    algo.alpha = alpha
    cv = cross_val_score(algo, 
                         X, 
                         y, 
                         scoring=criterion,
                         cv=K)

    # cv returns negative values, need abs()
    cv_score = np.mean(np.absolute(cv))
    cv_score_set.append(cv_score)

minimum_score = cv_score_set.index(np.min(cv_score_set))
alpha_value = parameter_set[minimum_score]

In [16]:
X_iris, y_iris = datasets.load_iris(return_X_y=True)

(150, 4)

### Resources and Notes: 

1. https://www.statology.org/k-fold-cross-validation-in-python/

In [67]:
# use this to view doc string
?tune_bb